In [1]:
import json
import cv2
from glob import glob
import re
import numpy as np
import os
import config
import pandas as pd
import preprocess as pp

In [2]:
df = pd.DataFrame(columns=['image_id', 'num', 'x', 'y', 'w', 'h', 'min', 'max', 'mean', 'std', 'diff'])

In [16]:
def preprocess_image(image):
    image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX)
    return image


In [17]:
def showRect(data, dictname, img, regions_i, origin_img, flaw_path, count, padding):
    flaw_path = f"{flaw_path.split('.')[-2]}_{str(count)}.{flaw_path.split('.')[-1]}"
    x_point = data[dictname]['regions'][regions_i]['shape_attributes']['x']
    y_point = data[dictname]['regions'][regions_i]['shape_attributes']['y']
    width =  data[dictname]['regions'][regions_i]['shape_attributes']['width']
    height =  data[dictname]['regions'][regions_i]['shape_attributes']['height']
    img = cv2.rectangle(img, (x_point, y_point, width, height), (0,0,255),2)
    try:
        y_point_1 = y_point - padding if y_point - padding > 0 else 0
        y_point_2 = y_point + height + padding if y_point + height + padding < origin_img.shape[0] else origin_img.shape[0]
        x_point_1 = x_point - padding if x_point - padding > 0 else 0
        x_point_2 = x_point + width + padding if x_point + width + padding < origin_img.shape[1] else origin_img.shape[1]
        grayscale_min = np.min(origin_img[y_point_1:y_point_2, x_point_1:x_point_2])
        grayscale_max = np.max(origin_img[y_point_1:y_point_2, x_point_1:x_point_2])
        grayscale_mean = np.mean(origin_img[y_point_1:y_point_2, x_point_1:x_point_2])
        grayscale_std = np.std(origin_img[y_point_1:y_point_2, x_point_1:x_point_2])
        grayscale_diff = grayscale_max - grayscale_min
        df.loc[len(df)] = [dictname, count, x_point, y_point, width, height, grayscale_min, 
                           grayscale_max, grayscale_mean, grayscale_std, grayscale_diff]
        #cv2.imwrite(flaw_path, origin_img[y_point_1:y_point_2, x_point_1:x_point_2])
        
    except Exception as e:
        print("Rect",e)
        print(flaw_path)
    return img, count + 1

def showCircle(data, dictname, img, regions_i, origin_img, flaw_path, count, padding):
    flaw_path = f"{flaw_path.split('.')[-2]}_{str(count)}.{flaw_path.split('.')[-1]}"
    x_center = data[dictname]['regions'][regions_i]['shape_attributes']['cx']
    y_center = data[dictname]['regions'][regions_i]['shape_attributes']['cy']
    radius = data[dictname]['regions'][regions_i]['shape_attributes']['r']
    
    img = cv2.circle(img, (x_center, y_center), radius, (0,0,225), 2)
    try:
        """y_point_1 = y_center - radius - padding if y_center - radius - padding > 0 else 0
        y_point_2 = y_center + radius + padding if y_center + radius + padding < origin_img.shape[0] else origin_img.shape[0]
        x_point_1 = x_center - radius - padding if x_center - radius - padding > 0 else 0
        x_point_2 = x_center + radius + padding if x_center + radius + padding < origin_img.shape[1] else origin_img.shape[1]
        grayscale_min = np.min(origin_img[y_point_1:y_point_2, x_point_1:x_point_2])
        grayscale_max = np.max(origin_img[y_point_1:y_point_2, x_point_1:x_point_2])
        grayscale_mean = np.mean(origin_img[y_point_1:y_point_2, x_point_1:x_point_2])
        grayscale_std = np.std(origin_img[y_point_1:y_point_2, x_point_1:x_point_2])
        grayscale_diff = grayscale_max - grayscale_min
        df.loc[len(df)] = [dictname, count, x_center, y_center, radius, radius, grayscale_min, grayscale_max, grayscale_mean, grayscale_std, grayscale_diff]
        cv2.imwrite(flaw_path, origin_img[y_point_1:y_point_2, x_point_1:x_point_2])"""
        
        
    except Exception as e:
        print("Circle",e)
        print(flaw_path)
    return img, count + 1

def showPolyline(data, dictname, img, regions_i, origin_img, flaw_path, count, padding):
    flaw_path = f"{flaw_path.split('.')[-2]}_{str(count)}.{flaw_path.split('.')[-1]}"
    x_points = data[dictname]['regions'][regions_i]['shape_attributes']['all_points_x']
    y_points = data[dictname]['regions'][regions_i]['shape_attributes']['all_points_y']
    pt = np.column_stack((x_points, y_points))
    
    img = cv2.polylines(img,[pt],True, (0,0,255),2)
    try:
        """y_point_1 = min(y_points) - padding if min(y_points) - padding > 0 else 0
        y_point_2 = max(y_points) + padding if max(y_points) + padding < origin_img.shape[0] else origin_img.shape[0]
        x_point_1 = min(x_points) - padding if min(x_points) - padding > 0 else 0
        x_point_2 = max(x_points) + padding if max(x_points) + padding < origin_img.shape[1] else origin_img.shape[1]
        grayscale_min = np.min(origin_img[y_point_1:y_point_2, x_point_1:x_point_2])
        grayscale_max = np.max(origin_img[y_point_1:y_point_2, x_point_1:x_point_2])
        grayscale_mean = np.mean(origin_img[y_point_1:y_point_2, x_point_1:x_point_2])
        grayscale_std = np.std(origin_img[y_point_1:y_point_2, x_point_1:x_point_2])
        grayscale_diff = grayscale_max - grayscale_min
        df.loc[len(df)] = [dictname, count, x_point_1, y_point_1, x_point_2 - x_point_1, y_point_2 - y_point_1, grayscale_min, grayscale_max, grayscale_mean, grayscale_std, grayscale_diff]
        cv2.imwrite(flaw_path, origin_img[y_point_1:y_point_2, x_point_1:x_point_2])"""
        
    except Exception as e:
        print("Polygone",e)
        print(flaw_path)
    return img, count + 1

def showEllipse(data, dictname, img, regions_i, origin_img, flaw_path, count, padding):
    flaw_path = f"{flaw_path.split('.')[-2]}_{str(count)}.{flaw_path.split('.')[-1]}"
    x_center = data[dictname]['regions'][regions_i]['shape_attributes']['cx']
    y_center = data[dictname]['regions'][regions_i]['shape_attributes']['cy']
    x_radius = data[dictname]['regions'][regions_i]['shape_attributes']['rx']
    y_radius = data[dictname]['regions'][regions_i]['shape_attributes']['ry']
    
    img = cv2.ellipse(img, (x_center, y_center), (x_radius, y_radius), 0, 0, 360, (0,0,225), 2)
    try:
        """y_point_1 = y_center - y_radius - padding if y_center - y_radius - padding > 0 else 0
        y_point_2 = y_center + y_radius + padding if y_center + y_radius + padding < origin_img.shape[0] else origin_img.shape[0]
        x_point_1 = x_center - x_radius - padding if x_center - x_radius - padding > 0 else 0
        x_point_2 = x_center + x_radius + padding if x_center + x_radius + padding < origin_img.shape[1] else origin_img.shape[1]
        grayscale_min = np.min(origin_img[y_point_1:y_point_2, x_point_1:x_point_2])
        grayscale_max = np.max(origin_img[y_point_1:y_point_2, x_point_1:x_point_2])
        grayscale_mean = np.mean(origin_img[y_point_1:y_point_2, x_point_1:x_point_2])
        grayscale_std = np.std(origin_img[y_point_1:y_point_2, x_point_1:x_point_2])
        grayscale_diff = grayscale_max - grayscale_min
        df.loc[len(df)] = [dictname, count, x_center, y_center, x_radius, y_radius, grayscale_min, grayscale_max, grayscale_mean, grayscale_std, grayscale_diff]
        cv2.imwrite(flaw_path, origin_img[y_point_1:y_point_2, x_point_1:x_point_2])"""
        
        
    except Exception as e:
        print("Ellipse",e)
        print(flaw_path)
    
    return img, count + 1

In [22]:
def showIMG(data, img_path, annotation_path, flaw_path, padding):
    """_summary_ 
    img_path에 있는 이미지를 어노테이션 정보(data)에 맞게 시각화하여 annotation_path에 저장하고, 결함 이미지를 flaw_path에 저장한다.

    Args:
        data (json): 파일 어노테이션 정보
        img_path (string): 이미지 경로
        annotation_path (string): 어노테이션 시각화한 이미지 저장 경로
        flaw_path (string): 결함 이미지 저장 경로
    """
    file_name = img_path.split('/')[-1]
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = preprocess_image(img)
    origin_img = img.copy()
    count = 0
    
    for fname in data.keys():
        if data[fname]['filename'] == file_name:
            dictname = fname
            for regions_i in range(len(data[dictname]['regions'])):
                regions_i = str(regions_i)
                shapetype = data[dictname]['regions'][regions_i]['shape_attributes']['name']
                if shapetype == 'rect':
                    img, count = showRect(data, dictname, img, regions_i, origin_img, flaw_path, count, padding)
                elif shapetype == 'circle':
                    img, count = showCircle(data, dictname, img, regions_i, origin_img, flaw_path, count, padding)
                elif shapetype == 'ellipse':
                    img, count = showEllipse(data, dictname, img, regions_i, origin_img, flaw_path, count, padding)
                elif shapetype == 'polygon':
                    img, count = showPolyline(data, dictname, img, regions_i, origin_img, flaw_path, count, padding)
                else:
                    pass
        cv2.imwrite(flaw_path, img)
        #cv2.imwrite(annotation_path, img)
        
    
            
 

In [23]:
flaw_category = glob(f"/home/VirtualFlaw/Data/Annotation/*")
for category in flaw_category:
    filelist = glob(f"{category}/label/*.json")
    for json_file in filelist:
        try:
            with open(json_file, 'r') as f:
                data = json.load(f)
            file_name = json_file.split('/')[-1].split('.')[0]
            file_category = file_name.split('-')[0] + '-' + file_name.split('-')[1] + '-' + file_name.split('-')[2]
            img_path = f"/home/VirtualFlaw/Data/Raw_jpg/202211/1REJ/{file_category}/{file_name}.jpg"
            padding = 10
            #save path
            annotation_path = config.ANNOTATION_DATA_PATH + category.split('/')[-1] + f'/{file_name}.jpg'
            flaw_path = "/home/VirtualFlaw/Hyun/Data/Annotation/"+f"{padding}pixel_padding/" + category.split('/')[-1] + f'/{file_name}.jpg'
            print(img_path)
            showIMG(data,img_path, annotation_path, flaw_path, padding)
        except Exception as e:
            print(e)
            print(json_file)
            continue
        
        

        
#df.to_csv(f"/home/VirtualFlaw/Hyun/src/info/flaw_padding_{padding}_info.csv", index=False)        


/home/VirtualFlaw/Data/Raw_jpg/202211/1REJ/22D017-07-023/22D017-07-023-1st-1-(2)-12-R-PO.jpg
/home/VirtualFlaw/Data/Raw_jpg/202211/1REJ/22D017-07-023/22D017-07-023-1st-1-(2)-10-R-PO.jpg
/home/VirtualFlaw/Data/Raw_jpg/202211/1REJ/22D017-07-023/22D017-07-023-1st-1-(2)-3-R-PO.jpg
/home/VirtualFlaw/Data/Raw_jpg/202211/1REJ/22D017-07-023/22D017-07-023-1st-1-(1)-51-R-PO.jpg
/home/VirtualFlaw/Data/Raw_jpg/202211/1REJ/22D017-07-023/22D017-07-023-1st-1-(2)-2-R-PO.jpg
/home/VirtualFlaw/Data/Raw_jpg/202211/1REJ/22D017-07-025/22D017-07-025-1st-1-(2)-35-R-PO.jpg
/home/VirtualFlaw/Data/Raw_jpg/202211/1REJ/22D017-07-023/22D017-07-023-1st-1-(2)-1-R-PO.jpg
/home/VirtualFlaw/Data/Raw_jpg/202211/1REJ/22D017-07-025/22D017-07-025-1st-1-(2)-33-R-PO.jpg
/home/VirtualFlaw/Data/Raw_jpg/202211/1REJ/22D017-07-023/22D017-07-023-1st-1-(2)-7-R-PO.jpg
/home/VirtualFlaw/Data/Raw_jpg/202211/1REJ/22D017-07-023/22D017-07-023-1st-1-(2)-6-R-PO.jpg
/home/VirtualFlaw/Data/Raw_jpg/202211/1REJ/22D017-07-025/22D017-07-025-1st-